In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re

from sklearn.linear_model import LinearRegression
import seaborn

In [5]:
pd.set_option('display.max_columns', 30)

### ここだけ手動で設定。変更がないか毎度確認すること

In [6]:
subject_name_list = ["kumakura","kim","souma","fujii","tubota","toki"]
phase_name_list = ["rest", "practice", "boredom", "flow", "ultra", "overload"]
target_columns = ["bpm", "ibi", "lf", "hf","lf/hf","FC3","FC4","FCz","mean_3ch","concentration_FC3","concentration_FC4","concentration_FCz","concentration_mean"]
target_phases = ["boredom", "flow","ultra","overload"]

folder_name = "regression_results_kubios_concentration"

### 分析対象のファイルのパスを正規表現で取得

In [7]:
pathes = glob.glob("/Users/miyakooti/repositories/arai_MATLAB_program/csv/?_*/HRV_and_PLI.csv")
pathes.sort()
pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/HRV_and_PLI.csv']

### フェーズごとの分析

In [10]:
for i,target_phase in enumerate(target_phases):
    
    for j,path in enumerate(pathes):
        if "kim" in path:
            continue
        df = pd.read_csv(path,index_col=0).fillna(0)
        # column
        # row
        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0:
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)
    
    export_data = {    
        "target": target_columns,
        "linear-p": [],
        "linear-rsquared": [],
        "linear-rsquared_adj": [],
        "linear-coef": [],
        "nonlinear-p": [],
        "nonlinear-rsquared": [],
        "nonlinear-rsquared_adj": [],
        "nonlinear-coef": [],
    }

    for target_column in target_columns:
        
        x = flow_dataset[target_column]
        y = flow_dataset[["questionnaire_average"]]
        
        for k in ["linear", "nonlinear"]:
            if k=="linear":
                # 線形単回帰
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["linear-p"].append(results.pvalues[target_column])
                export_data["linear-rsquared"].append(results.rsquared)
                export_data["linear-rsquared_adj"].append(results.rsquared_adj)
                export_data["linear-coef"].append(results.params[target_column])
            if k=="nonlinear":
                # ２次単回帰
                x = x**2
                
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["nonlinear-p"].append(results.pvalues[target_column])
                export_data["nonlinear-rsquared"].append(results.rsquared)
                export_data["nonlinear-rsquared_adj"].append(results.rsquared_adj)
                export_data["nonlinear-coef"].append(results.params[target_column])
    
    print(f"\nsaving {target_phase} phase regression results...\n")
    df = pd.DataFrame(export_data)   
    print(df)
    save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{folder_name}/{i}_{target_phase}_regression.csv"
    df.to_csv(save_path)  


saving boredom phase regression results...

                target  linear-p  linear-rsquared  linear-rsquared_adj  \
0                  bpm  0.722303         0.048351            -0.268866   
1                  ibi  0.664789         0.071002            -0.238664   
2                   lf  0.789557         0.027572            -0.296571   
3                   hf  0.342112         0.297072             0.062762   
4                lf/hf  0.377047         0.262872             0.017163   
5                  FC3  0.314522         0.326222             0.101629   
6                  FC4  0.246629         0.407220             0.209626   
7                  FCz  0.669641         0.068911            -0.241451   
8             mean_3ch  0.372154         0.267489             0.023319   
9    concentration_FC3  0.407174         0.235617            -0.019178   
10   concentration_FC4  0.552739         0.128993            -0.161342   
11   concentration_FCz  0.522268         0.148179            -0.135